<style>
    @media print{
        body {
            position:relative !important;
        }
        .celltag_new_page {
            page-break-before: always !important;
        }
    }
</style>
# COMPSCI 371 Homework 7

_**Group Members:**_ Ethan Goldberg, Noah Costa, Charlie Oestreicher

### Problem 0 (3 points)

## Part 1: The Bootstrap Sampler

### Problem 1.1

In [4]:
import numpy as np

def sample(n, shape, gen):
    # yes, this function could be one line,
    # but this is far more readable.
    m = np.prod(shape)
    random_integers = gen.choice(n, size=m, replace=True)
    result = random_integers.reshape(shape)
    return result

rng = np.random.default_rng(15)
sample(5, (2, 7), rng)

array([[4, 3, 3, 4, 1, 1, 1],
       [0, 2, 2, 4, 0, 4, 3]])

### Problem 1.2 (Exam Style)

$V_1 = \{0,1,5,6,8\}$

### Problem 1.3

In [20]:
import numpy as np

def generate_bootstrap_validation_sets(N, R):
    """
    Generate R validation sets using bootstrap resampling on a training set of size N.
    
    Args:
    N (int): Size of the training set.
    R (int): Number of repetitions.

    Returns:
    List of R validation sets, each represented as a NumPy array.
    """
    training_set = np.arange(N)  # Create the original training set.

    validation_sets = []
    for _ in range(R):
        # Generate a validation set by sampling N data points with replacement.
        validation_set = np.random.choice(training_set, N, replace=True)
        validation_sets.append(validation_set)

    return validation_sets

# Example usage:
N = 100  # Size of the training set
R = 5    # Number of repetitions
validation_sets = generate_bootstrap_validation_sets(N, R)

for i, validation_set in enumerate(validation_sets):
    print(f"Validation Set {i + 1}: {validation_set}")



Validation Set 1: [46 26 58 83  4  1 49 83 66 64  8 46 70 80 63 35 70 20 43 52 46 76 18 34
 39 26 62  0 54 27 93 97 99 92 43 75 11 57 77 59 25 58 11 15 15 91 45 72
 84 97 49 23 19 24 84 53  7 54 72 91 98 60 34 42 45 13 66 83 71 78 22  6
 89 67 27 44 35 45 72 58 35 41 74 82 83 48 73  1 33 49 50  5 92 20 87 74
 23 61 24 19]
Validation Set 2: [54 65  6 53 53 40 71 57 87 50 92 10 31 41 40 52 64 25  2  2 48 21 70 96
 66 42 22 71  1 72 13 14 23 53 42 76 90 14 22 45 36 73  8 21 77  1 16 34
  4 72 11 88 19 77 98  4 94 46 38  7 33 54 96 73  0 97 87 93 39  8  9 21
 71 59 20 28 60 39 91 24 16  2 11 53 67  8 89 21 17 94 70  6  5 90 34 42
 11 58 95 53]
Validation Set 3: [40 11  9 27 35 94 19 81 95 35 69 66 79 53 84 18 55 85 88 81 35  0 45  2
 17 78 50 35 16 36 45 47 87 90 83 40 24 41 34  3 80 79 54 72  0 36 84 57
 64 22 43 98 87 50 40 32 16 77 17 86 92 43 60 93 51 47 57 84 69 37 11 51
 56 36 50 54 47  8 94 46 59 66 51 12 24 78 34 25 96 83 41 26 98 97 73 47
  7 75 32 63]
Validation Set 4: [45 75 31 

### Problem 1.4

### Problem 1.5 (Exam Style)

## Part 2: Cross-Validation with the Bootstrap

### Problem 2.1

In [7]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from matplotlib import pyplot as plt
from time import time
from tqdm import tqdm

In [8]:
def experiment(c_range=np.logspace(-3, 3, num=20), n_folds=5):
    rng = np.random.default_rng(15)
    start = time()
    clf, val_acc, val_acc_variance, c_acc = bootstrap_validation(
        mnist.train, c_range, n_folds, rng
    )
    stop = time()
    test_accuracy = clf.score(mnist.test.x, mnist.test.y)

    print('time for bootstrap validation {:.1f} seconds'.format(stop - start))
    plt.figure()
    plt.semilogx(c_range, c_acc)
    plt.xlabel('C'), plt.ylabel('validation accuracy')
    print('best c: {:.4g}'.format(clf.C))
    print('validation accuracy and variance: {:.4g}, {:.4g}'.format(val_acc, val_acc_variance))
    print('test accuracy {:.4f}'.format(test_accuracy))
    plt.show()

In [9]:
import urllib.request
import ssl
from os import path as osp
import shutil

In [10]:
def retrieve(file_name, semester='fall23', homework=7):
    if osp.exists(file_name):
        print('Using previously downloaded file {}'.format(file_name))
    else:
        context = ssl._create_unverified_context()
        fmt = 'https://www2.cs.duke.edu/courses/{}/compsci371/homework/{}/{}'
        url = fmt.format(semester, homework, file_name)
        with urllib.request.urlopen(url, context=context) as response:
            with open(file_name, 'wb') as file:
                shutil.copyfileobj(response, file)
        print('Downloaded file {}'.format(file_name))

In [11]:
import pickle

file_name = 'mnist.pickle'
retrieve(file_name, homework=2)

with open(file_name, 'rb') as file:
    mnist = pickle.load(file)

Using previously downloaded file mnist.pickle


### Problem 2.2 (Exam Style)

## Part 3: Basics of Linear Support Vector Machines 

### Problem 3.1 (Exam Style)

## Part 4: Binary Linear SVMs and LRCs

### Problem 4.1 (Exam Style)

### Problem 4.2

In [14]:
balanced_file_name = 'balanced.pickle'
retrieve(balanced_file_name)

Using previously downloaded file balanced.pickle


In [15]:
with open(balanced_file_name, 'rb') as file:
    balanced_data = pickle.load(file)

In [16]:
def show_data(data, title):
    fs = 12
    x, y = data.x, data.y
    labels, colors = (-1, 1), ('r', 'b')
    for label, color in zip(labels, colors):
        k = y == label
        plt.plot(x[k, 0], x[k, 1], 'o' + color,
                 label='y = {}'.format(label), ms=3)
    plt.legend(fontsize=fs)
    plt.axis('equal')
    plt.axis('off')
    plt.title(title, fontsize=fs)

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline


def draw_line(b, w, style):
    xlim, ylim = plt.xlim(), plt.ylim()
    if np.abs(w[0]) > np.abs(w[1]):
        xs = np.array(xlim) + np.array([-1, 1])
        ys = (-b - w[0] * xs) / w[1]
    else:
        ys = np.array(ylim) + np.array([-1, 1])
        xs = (-b - w[1] * ys) / w[0]
    plt.plot(xs, ys, style, c='k')
    plt.xlim(xlim)
    plt.ylim(ylim)

Comments:
- (i) The datasets (train and test) are not linearly separable, and accuracy is therefore less than 100 percent.
- (ii) Both classifiers generalize well, and in fact achieve testing accuracy that is slightly _greater_ than their training accuracy.
- (iii) The two classifiers achieve equal performance on both the training set and the test set. Visually, they appear to compute the exact same boundary.

### Problem 4.3

In [25]:
imbalanced_file_name = 'imbalanced.pickle'
retrieve(imbalanced_file_name)

Using previously downloaded file imbalanced.pickle


In [26]:
with open(imbalanced_file_name, 'rb') as file:
    imbalanced_data = pickle.load(file)